# C/C++ DLL with Python and Excel VBA

## 1 Using C/C++ DLL with Python

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. 

But for certain types of calculations Python (and any other interpreted language) can be **very slow**.

Such calculations may be implemented in a compiled language such as C or Fortran.

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

### 1.1 Compiler a shared library with `multiple` files

The shared library with `multiple` code files

* fibonacci.h/c

* funs.h/c

### Once-Only Headers

If a header file happens to be included twice, the compiler will process its contents twice.
This is very likely to cause an error, e.g. when the compiler sees the same structure definition
twice. Even if it does not, it will certainly waste time.

The standard way to prevent this is to enclose the entire real contents of the file in a
conditional, like this:
```c
#ifndef FUNS_H
#define FUNS_H

the entire file

#endif /* !FUNS_H */
```
This construct is commonly known as a wrapper **#ifndef**. When the header is included
again, the conditional will be false, because **FUNS_H** is defined. The preprocessor
will skip over the entire contents of the file, and the compiler will not see it twice.

In [ ]:
%%file ./code/gcc/funs.h

#ifndef FUNS_H
#define FUNS_H

void hello();
double dprod(double *x, int n);
unsigned long LinearFibonacci(int n);

#endif

In [ ]:
%%file ./code/gcc/funs.c

#include <stdio.h>
#include "funs.h"

void hello()
{
   printf("C says Hello world!\n");
}

// x[0]*x[1]*...*x[n-1]
double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

// The linear time algorithm for finding Fibonacci numbers
//   females(0) =1
//   females(1) = 1
//   females(n + 2) = females(n+1) + females(n）
// n------------                                             
//   f1=  females(n）
//   f2=  females(n+1)
//   fib = f1 + f2;
//   
//    f2=f1; ->females(n+1)
//    f1=fib; ->females(n + 2) 
//n+1----------
//    f1=  females(n+1）
//    f2=  females(n+1+1)
//    fib = f1 + f2;              
//                  
unsigned long LinearFibonacci(int n)
{
  unsigned long fib;
  if (n< 0)
      return -1;
  
  // base case 0 or 1
  if (n == 0||n==1)
      return 1;
 
  unsigned long f1=1,f2 = 1;
  for(int i=2; i<=n; i++)
  {
    // 1 only add
    fib = f1 + f2; 
    // 2 swap  
    f2=f1; 
    f1=fib;
  }    
  return fib;
}

The **linear time algorithm** for finding `Fibonacci numbers` 

http://www.catonmat.net/blog/on-the-linear-time-algorithm-for-finding-fibonacci-numbers/

In [1]:
!gcc -c -O3 -Wall -fPIC -o funs.o ./code/gcc/funs.c 
!gcc -c -O3 -Wall -fPIC -o fibonacci.o ./code/gcc/fibonacci.c
!gcc -shared -o ./code/gcc/libmultifuns.dll  funs.o  fibonacci.o

In [2]:
!gcc -c -O3 -Wall -fPIC  ./code/gcc/funs.c ./code/gcc/fibonacci.c
!gcc -shared -o ./code/gcc/libmultifuns.dll  funs.o  fibonacci.o

In [3]:
!dir .\code\gcc\libmulti*.dll

 驱动器 D 中的卷是 cmh
 卷的序列号是 02AB-E07E

 D:\SEUCourse\SE_ThermalEnergy\S2017\home\notebook\code\gcc 的目录

2017/05/16  14:09            49,173 libmultifuns.dll
               1 个文件         49,173 字节
               0 个目录 200,843,644,928 可用字节


In [4]:
%%file makefile

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 gcc  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
	 del funs.o fibonacci.o
    
multifunsobj: ./code/gcc/funs.c ./code/gcc/fibonacci.c
	 gcc -c -O3 -Wall -fPIC ./code/gcc/funs.c ./code/gcc/fibonacci.c
     
clean:
	 del .\code\gcc\libmultifuns.dll

Overwriting makefile


In [5]:
!make

gcc -c -O3 -Wall -fPIC ./code/gcc/funs.c ./code/gcc/fibonacci.c
gcc  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
del funs.o fibonacci.o


In [ ]:
!make clean

The result is a compiled shared library **`libmultifuns.dll`**

### Using the `variable` in  makefile

A `variable` begins with a **$** and is enclosed within parentheses **(...)** 

In [ ]:
%%file makefile

CC=gcc
CFLAGS=-O3 -Wall -fPIC  

INC = -I ./code/gcc/ 

SRCS= ./code/gcc/funs.c ./code/gcc/fibonacci.c 

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC)  -shared -o ./code/gcc/libmultifuns.dll funs.o fibonacci.o
	 del funs.o fibonacci.o
    
multifunsobj: 
	 $(CC) -c $(CFLAGS) $(INC) $(SRCS) 
     
clean:
	 del .\code\gcc\libmultifuns.dll

In [ ]:
!make

In [ ]:
!make clean

### 1.2 ctypes - access the C library

**ctypes** is a foreign function library for Python. It provides C compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

http://docs.python.org/3/library/ctypes.html

We need to load the library and set properties such as the functions return and argument types using the **ctypes** package :

* **ctypes** exports the **cdll**, and on Windows **windll** objects, for loading dynamic link libraries.
   * **cdll.LoadLibrary(name)** : loads the library  which export functions using standard `__cdecl` calling convention
   * **windll.LoadLibrary(name)** : loads the library with `__stdcall` calling convention for the function 


* **argtypes** : the types of the arguments 


* **restype**: the types of return values. 

#### Example `__stdcall` calling convention for the function(windows only) 

In [6]:
from ctypes import *
print(windll.kernel32)  

<WinDLL 'kernel32', handle 7ffa08020000 at 0x186404f19e8>


#### Example `__cdecl` calling convention for the function(windows,Linux)

**msvcrt** is the MS **standard C library** containing most standard C functions, and uses the cdecl calling convention:

In [7]:
print(cdll.msvcrt)

<CDLL 'msvcrt', handle 7ffa07420000 at 0x18643721198>


### Wrap libmultifuns.dll in pure Python.

__cdecl calling convention

In [8]:
%%file ./code/gcc/multifuns.py

from ctypes import cdll,c_void_p,c_int,c_long,c_double,POINTER,byref
import numpy as np


_lib = cdll.LoadLibrary('./code/gcc/libmultifuns.dll')

# void hello()
def hello():
    _lib.hello.argtypes = []
    _lib.hello.restype  =  c_void_p
    return _lib.hello()

# double dprod(double *x, int n)
def dprod(x):
    _lib.dprod.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), c_int]
    _lib.dprod.restype  = c_double
    n = len(x)
    x = np.asarray(x, dtype=np.float)
    return _lib.dprod(x, int(n))

#unsigned long fibonacci(int n, unsigned long *fib_cache);
def LinearFibonacci(n):
    _lib.LinearFibonacci.argtypes = [c_int]
    _lib.LinearFibonacci.restype= c_long
    return _lib.LinearFibonacci(int(n))

#unsigned long fibonacci(int n, unsigned long *fib_cache);
def fibonacci(n):
    fibcache = (POINTER(c_long)*n)()
    fib=c_long()
    fib=_lib.LinearFibonacci(int(n),byref(fibcache))
    return  fib

Overwriting ./code/gcc/multifuns.py


In [9]:
%%file ./code/gcc/run_hello_c.py

import multifuns
multifuns.hello()

Overwriting ./code/gcc/run_hello_c.py


In [10]:
!python .\code\gcc\run_hello_c.py

C says Hello world!


### Product function:

In [12]:
import sys
sys.path.append('./code/gcc')
import multifuns
multifuns.dprod([12,3,4,5,6]) 

4320.0

### The linear time algorithm for finding Fibonacci numbers

In [13]:
import sys
sys.path.append('./code/gcc')
import multifuns
multifuns.LinearFibonacci(10) 

89

###  Fibonacci 

In [14]:
import sys
sys.path.append('./code/gcc')
import multifuns
multifuns.fibonacci(10)

89

### QUIZ:Timing Fibonacci Methods

`Fibonacci in C` and `Fibonacci in Python`

* Which one is the fastest? 

* Why? 

* What message does this give us?

#### 1 Fibonacci in C

In [ ]:
%timeit multifuns.LinearFibonacci(50) 

In [ ]:
%timeit multifuns.fibonacci(50) 

#### 2 LinearFibonacci in Python

In [ ]:
def LinearFibonacci(n):
    curr, next = 1, 1
    for i in range(2, n+1):
        next, curr = ((next+curr),next)
    return next

In [ ]:
%timeit LinearFibonacci(50) 

In [ ]:
#Page 254, Figure 18.3
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
       Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

In [ ]:
%timeit fastFib(50) 

## 2 C/C++ DLL for Visual Basic Application with Excel

  For `Visual Basic` applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
VBA can **only** call `__stdcall` functions, not `__cdecl` functions. 
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an `exported` function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

#### Step 1: Create your DLL with `__stdcall` calling convention

Create a DLL with the following code:

* mathfuns.h

* mathfuns.c

Export all functions as `__stdcall`.

In [15]:
%%file ./code/gcc/mathfuns.h

#ifdef BUILD_DLL
#define DLLPORT __declspec(dllexport)
#else
#define DLLPORT __declspec(dllimport)
#endif

DLLPORT __stdcall double fadd(double a,double b);
DLLPORT __stdcall double fmul(double a, double b);

Overwriting ./code/gcc/mathfuns.h


When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the `export` directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [16]:
%%file ./code/gcc/mathfuns.c

#include "mathfuns.h"

DLLPORT __stdcall double fadd(double a,double b){
   return (a+b);
}

DLLPORT __stdcall double  fmul(double a, double b){
   return (a*b);
}

Overwriting ./code/gcc/mathfuns.c


In [17]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/gcc/libmathfuns.dll mathfuns.o -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: `Predefine name as a macro`, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   This adds an undecorated alias for the `exported function names` that is simply **the name of the function** 

In [18]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o ./code/gcc/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=./code/gcc/libmathfuns.def

* -static-libgcc

   This option links the GNU `libgcc` library `statically`. 
   

* -output-def=libmathfuns.def

    Name of `.def` file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [ ]:
# %load ./code/gcc/libmathfuns.def
EXPORTS
    fadd @1
    fmul @2


In [ ]:
%%file makefile

CC=gcc
CFLAGS=-DBUILD_DLL -o

all: libmathfuns.dll

libmathfuns.dll: mathfunsobj
	 $(CC) -shared -o ./code/gcc/libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
	 del mathfuns.o
    
mathfunsobj: ./code/gcc/mathfuns.c
	 $(CC) -c $(CFLAGS) mathfuns.o ./code/gcc/mathfuns.c
     
clean:
	 del .\code\gcc\libmathfuns.dll

In [ ]:
!make

### 2 call_stdcall DLL from Python

```python
 windll.LoadLibrary
```

In [ ]:
%%file ./code/gcc/mathfuns.py

from ctypes import windll,c_double

flib = windll.LoadLibrary('./code/gcc/libmathfuns.dll')


def fadd(a,b):
    flib.fadd.argtypes = [c_double,c_double]
    flib.fadd.restype  = c_double
    return flib.fadd(a,b)

def fmul(a,b):
    flib.fmul.argtypes = [c_double,c_double]
    flib.fmul.restype  = c_double
    return flib.fmul(a,b)

In [ ]:
import sys
sys.path.append('./code/gcc')
from mathfuns import *
a=4.6
b=5
print(a,'+',b,'=',fadd(a,b))
print(a,'*',b,'=',fmul(a,b))

#### SEUIF97：call_stdcall DLL from Python

WINFUNCTYPE

In [ ]:
from ctypes import *

flib = windll.LoadLibrary('libseuif97.dll')
prototype = WINFUNCTYPE(c_double, c_double, c_double, c_int)

# ---(p,t) ----------------

def pt(p, t, pid):
    f = prototype(("seupt", flib),)
    result = f(p, t, pid)
    return result


def pt2h(p, t):
    f = prototype(("seupt", flib),)
    result = f(p, t, 4)
    return result

### 3 Call __stdcall DLL from C 

In [20]:
%%file ./code/gcc/mainmathfuns.c

#include <stdio.h>
#include "mathfuns.h"
 
int main() {
    double a=5.3;
    double b=6.1;
    double radd=fadd(a,b);
    double rmul=fmul(a,b);
    printf("%.3f + %.3f = %.3f \n",a,b,radd);
    printf("%.3f * %.3f = %.3f \n",a,b,rmul);
    return 0;
}

Overwriting ./code/gcc/mainmathfuns.c


In [21]:
!gcc -c -o mainmathfuns.o ./code/gcc/mainmathfuns.c 
!gcc -o  ./code/gcc/mainmathfuns.exe mainmathfuns.o -I./code/gcc/ -L./code/gcc/ -lmathfuns

In [22]:
!.\code\gcc\mainmathfuns

5.300 + 6.100 = 11.400 
5.300 * 6.100 = 32.330 


In [ ]:
%%file makefile

all: mainmathfuns.exe

clean:
	del .\code\gcc\mainmathfuns.exe

mainmathfuns.exe: mainmathfuns.o ./code/gcc/mathfuns.h 
	gcc -o ./code/gcc/mainmathfuns mainmathfuns.o -I./code/gcc -L./code/gcc/ -lmathfuns
	del *.o

mainmathfuns.o: ./code/gcc/mainmathfuns.c 
	gcc -c ./code/gcc/mainmathfuns.c 

In [ ]:
!make

In [ ]:
!.\code\gcc\mainmathfuns

In [ ]:
!make clean

### Step 4: Call DLL from Excel VBA

**1** `libmathfuns.dll` in the default dll path of windows `C:\windows\system`


**2** demo-vba.xlsm in ./code/gcc/


**3**  press "ALT+F11" into VBA

* 3.1 create the module **mathfuns** to library call such as:

```vba
Public Declare PtrSafe Function fadd Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
Public Declare PtrSafe Function fmul Lib "libmathfuns" (ByVal a As Double, ByVal b As Double) As Double
```
* 3.2 create the module **vbaapi** to call such as:

```vba
Public Function sadd(ByVal a As Double, ByVal b As Double) As Double
    sadd = mathfuns.fadd(a, b)
End Function
 
Public Function smul(ByVal a As Double, ByVal b As Double) As Double
    smul = mathfuns.fmul(a, b)
End Function
```

**4**  call in cells
![demo-vba](./code/gcc/demo-vba.jpg)

### Further reading

* The linear time algorithm for finding Fibonacci numbers http://www.catonmat.net/blog/on-the-linear-time-algorithm-for-finding-fibonacci-numbers/


* ctypes

   * http://docs.python.org/3/library/ctypes.html

   * http://scipy-cookbook.readthedocs.io/items/Ctypes.html

   * https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html


* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


* Getting Started with VBA in Office 2010: https://msdn.microsoft.com/library/office/ee814735(v=office.14)
   
* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html


* Excel4Engineering https://github.com/PySEE/Excel4Engineering